In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nltk import RegexpTokenizer, pos_tag, sent_tokenize, word_tokenize
from pandas.io.json import json_normalize

import json
import pandas as pd
import re

In [3]:
data = []
with open('dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
            
            data.append([sentence for sentence in sent_tokenize(paragraph)])

# Sentence Selection

### Sentence importance sorting

In [4]:
idx = 143

for i in range(len(data[idx])):
    print(i, data[idx][i])

0 John was born to Henry II of England and Eleanor of Aquitaine on 24 December 1166.
1 Henry had inherited significant territories along the Atlantic seaboard—Anjou, Normandy and England—and expanded his empire by conquering Brittany.
2 Henry married the powerful Eleanor of Aquitaine, who reigned over the Duchy of Aquitaine and had a tenuous claim to Toulouse and Auvergne in southern France, in addition to being the former wife of Louis VII of France.
3 The result was the Angevin Empire, named after Henry's paternal title as Count of Anjou and, more specifically, its seat in Angers.
4 [nb 2] The Empire, however, was inherently fragile: although all the lands owed allegiance to Henry, the disparate parts each had their own histories, traditions and governance structures.
5 As one moved south through Anjou and Aquitaine, the extent of Henry's power in the provinces diminished considerably, scarcely resembling the modern concept of an empire at all.
6 Some of the traditional ties between 

In [5]:
from sentence_selection import MultiWordPhraseExtractor, preprocess

rank = MultiWordPhraseExtractor(window_size=5, top_keywords=7).summarize(preprocess(' '.join(data[idx])), top=3)
rank

[1, 0, 6]

# Sentence Parsing

In [6]:
from question_generation import Sentence

Load Model Success
****************************************************************************************************
default values:
senna path:
 D:\Programming\question-answer-generation\QuestionGeneration\lib\senna\ 
Dependencie parser:
 D:\Programming\question-answer-generation\QuestionGeneration\lib\
Stanford parser clr java -cp stanford-parser.jar edu.stanford.nlp.trees.EnglishGrammaticalStructure -treeFile in.parse -collapsed
****************************************************************************************************


In [7]:
# sentence = Sentence(data[idx][0])
sentence = Sentence("The building was on fire, and it wasn't my fault.")
sentence.sentence

"The building was on fire, and it wasn't my fault."

In [8]:
sentence.parse_constituents()
print(list(map(lambda x: print(x), sentence.constituent_tree)))

(S
  (S
    (NP (DT The) (NN building))
    (VP (VBD was) (PP (IN on) (NP (NN fire)))))
  (, ,)
  (CC and)
  (S (NP (PRP it)) (VP (VBD was) (RB n't) (NP (PRP$ my) (NN fault))))
  (. .))
[None]


In [9]:
sentence.extract_named_entities()
sentence.ner_classes

[('The', 'O'),
 ('building', 'O'),
 ('was', 'O'),
 ('on', 'O'),
 ('fire', 'O'),
 ('and', 'O'),
 ('it', 'O'),
 ('wasn', 'O'),
 ('t', 'O'),
 ('my', 'O'),
 ('fault', 'O')]

In [10]:
# sentence.parse_srl()
# for clause in sentence.srl:
#     print(clause)

# Gap Formulation

In [11]:
from question_generation import GapSelector

In [12]:
gapper = GapSelector(sentence)

candidates = gapper.process()
for i, parent in enumerate(gapper.parents):
    print(i, parent)

0 (S
  (NP (DT The) (NN building))
  (VP (VBD was) (PP (IN on) (NP (NN fire)))))
1 (S
  (NP (DT The) (NN building))
  (VP (VBD was) (PP (IN on) (NP (NN fire)))))
2 (S (NP (PRP it)) (VP (VBD was) (RB n't) (NP (PRP$ my) (NN fault))))
3 (S (NP (PRP it)) (VP (VBD was) (RB n't) (NP (PRP$ my) (NN fault))))


In [13]:
print("Sentence: {}\n".format(sentence.sentence))
for candidate in candidates:
    print("Q: {}\nA: {}\n".format(candidate['question'], candidate['answer']))

Sentence: The building was on fire, and it wasn't my fault.

Q: _____ was on fire
A: The building

Q: The building was on _____
A: fire

Q: _____ wasn't my fault
A: it

Q: it wasn't _____
A: my fault



In [18]:
from simple_nlg import SimpleNLG

for candidate in gapper.candidates:
    nlg = SimpleNLG(sentence, candidate)
    question = nlg.convert_to_question()
    
    print("Q: {}\nA: {}\n".format(question, candidate['answer']))

SS
Q: What was on fire?
A: The building

PP
Q: The building The building?
A: fire

SS
Q: What wasn't my fault?
A: it

OO
NOT
Q: 
A: my fault



In [15]:
# def check_in_dict(answer, ne_dict, key):
#     if key not in ne_dict:
#         return False
#     if answer in ne_dict[key]:
#         return True
    
#     for word in answer.split(' '):
#         if word in ne_dict[key]:
#             return True
#     return False    

# from nltk.stem import WordNetLemmatizer

# from simple_nlg import init_simple_nlg

# gateway = init_simple_nlg()
# lexicon = gateway.jvm.Lexicon.getDefaultLexicon()
# nlg_factory = gateway.jvm.NLGFactory(lexicon)
# realiser = gateway.jvm.Realiser(lexicon)

# wnl = WordNetLemmatizer()

# blank = '_____'

# for candidate in gapper.candidates:
#     entity = candidate['entity']
#     parent = candidate['parent']
#     answer = candidate['answer']
#     question = candidate['question']
    
#     # Subject
#     if entity.parent().label() == 'S':
#         if 'PER' in sentence.named_entities_dict and answer in sentence.named_entities_dict['PER']:
#             question = question.replace(blank, "who")
#         else:
#             question = question.replace(blank, "what")
        
#         q = list(question)
#         q[0] = q[0].upper()
#         if q[-1] == '.':
#             q[-1] = '?'
#         else:
#             q.append('?')
#         question = ''.join(q)
        
#     # Object
#     elif entity.parent().parent().label() == 'S':
#         p = nlg_factory.createClause()
#         p.setObject(answer)
        
#         for child in parent:
#             # Subject
#             if child.label() == 'NP':
#                 p.setSubject(' '.join(child.leaves()))
            
#             # Verb
#             elif child.label() == 'VP':
#                 for gchild in child:
#                     if gchild.label() in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
#                         p.setVerb(wnl.lemmatize(gchild.leaves()[0], pos='v'))
#                         if gchild.label() in ['VB', 'VBG', 'VBP', 'VBZ']:
#                             p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PRESENT)
#                         elif gchild.label() in ['VBD', 'VBN']:
#                             p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PAST)
#                     elif gchild.label() == 'RB':
#                         if gchild.leaves()[0] in ['n\'t', 'not']:
#                             p.setFeature(gateway.jvm.Feature.NEGATED, True)
                        
#             elif child.label() == 'PP':
#                 p.addComplement(' '.join(child.leaves()))
                        
#         if 'PER' in sentence.named_entities_dict and answer in sentence.named_entities_dict['PER']:
#             p.setFeature(gateway.jvm.Feature.INTERROGATIVE_TYPE, gateway.jvm.InterrogativeType.WHO_OBJECT)
#         else:
#             p.setFeature(gateway.jvm.Feature.INTERROGATIVE_TYPE, gateway.jvm.InterrogativeType.WHAT_OBJECT)
            
#         question = realiser.realiseSentence(p)
        
#     elif entity.parent().label() == 'PP':
#         p = nlg_factory.createClause()
        
#         for child in parent:
#             # Subject
#             if child.label() == 'NP':
#                 p.setSubject(' '.join(child.leaves()))
            
#             # Verb
#             elif child.label() == 'VP':
#                 for gchild in child:
#                     if gchild.label() in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
#                         p.setVerb(wnl.lemmatize(gchild.leaves()[0], pos='v'))
#                         if gchild.label() in ['VB', 'VBG', 'VBP', 'VBZ']:
#                             p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PRESENT)
#                         elif gchild.label() in ['VBD', 'VBN']:
#                             p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PAST)
#                     elif gchild.label() == 'NP':
#                         p.setObject(' '.join(gchild.leaves()))
#                     elif gchild.label() == 'RB':
#                         if gchild.leaves()[0] in ['n\'t', 'not']:
#                             p.setFeature(gateway.jvm.Feature.NEGATED, True)                        
#             elif child.label() == 'PP':
#                 p.addComplement(' '.join(child.leaves()))
                
#         p.setFeature(gateway.jvm.Feature.INTERROGATIVE_TYPE, gateway.jvm.InterrogativeType.WHO_INDIRECT_OBJECT)
#         question = realiser.realiseSentence(p)\
#             .replace(answer, '')\
#             .replace('  ', ' ')\
#             .replace('to?', '?')\
#             .replace(' ?', '?')
#         question = re.sub(r"-LRB- (.*) -RRB-", r"(\1)", question)
        
#         if check_in_dict(answer, sentence.named_entities_dict, 'PER'):
#             pass
#         elif check_in_dict(answer, sentence.named_entities_dict, 'LOC'):
#             question = question.replace('Who', 'Where')
#         else:
#             question = question.replace('Who', 'What')
            
#     print("Q: {}\nA: {}\n".format(question, candidate['answer']))
        